In [40]:
import datetime as dt
import pandas as pd
import numpy as np
from numpy import arange
import matplotlib.pyplot as plt
from pandas import read_csv
from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import RandomizedSearchCV


import warnings
warnings.simplefilter(action='ignore')

In [41]:
df = pd.read_excel(r"C:\treinamento_skit_learn\Computed Prices With Metadata.xlsx")
df.drop_duplicates(inplace=True)
#df.set_index("Date", inplace=True)
df['Valid Until'] = pd.to_datetime(df['Valid Until'],errors='coerce')

df['EAN (View Only)'] = df['EAN (View Only)'].astype(str).apply(lambda k: str(k).split(".0")[0])

produtos = df[['SKU ID', 'Trade Policy', 'List Price', 'Price', 'Cost Price',
        'Product ID (View Only)']]
       
produtos[['List Price','Price','Cost Price']] = produtos[['List Price','Price','Cost Price']].applymap(lambda k: float(str(k).replace(".","").replace(",","."))).round(2)

In [42]:
produtos.dropna(inplace=True)

In [43]:

y = produtos['Price']
x = produtos.drop(['Price'],axis=1)
x

,SKU ID,Trade Policy,List Price,Cost Price,Product ID (View Only)
0,129,1,199.0,197.0,129
1,129,2,199.0,197.0,129
2,129,4,27937.0,197.0,129
3,129,6,299.0,197.0,129
4,129,10,299.0,197.0,129
...,...,...,...,...,...
6732,791,10,13936.0,9743.0,791
6733,791,13,13936.0,9743.0,791
6734,791,14,13936.0,9743.0,791
6735,791,15,13936.0,9743.0,791


In [44]:
print(x)

      SKU ID  Trade Policy  List Price  Cost Price  Product ID (View Only)
0        129             1       199.0       197.0                     129
1        129             2       199.0       197.0                     129
2        129             4     27937.0       197.0                     129
3        129             6       299.0       197.0                     129
4        129            10       299.0       197.0                     129
...      ...           ...         ...         ...                     ...
6732     791            10     13936.0      9743.0                     791
6733     791            13     13936.0      9743.0                     791
6734     791            14     13936.0      9743.0                     791
6735     791            15     13936.0      9743.0                     791
6736     791            17     13936.0      9743.0                     791

[6737 rows x 5 columns]


In [45]:
"""Separa modelos"""
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.33,  random_state=0)

In [46]:
scale = StandardScaler()
x_train = scale.fit_transform(x_train)
x_test = scale.transform(x_test)

In [47]:
x_test

array([[-0.25573092, -1.23796222, -0.23996197, -0.15674395, -0.2528318 ],
       [ 0.46283449,  1.39845684, -0.22998259, -0.25268388,  0.46662098],
       [-0.72590334,  1.39845684, -0.17759088, -0.03948403, -0.72358485],
       ...,
       [ 0.14790768,  0.16812794,  0.00919806, -0.61322528,  0.15130526],
       [ 0.88865103,  0.69541175,  0.09644274,  1.08492617,  0.89296337],
       [ 0.40073624,  1.39845684, -0.00507244,  0.66831029,  0.40444605]])

In [48]:
"""Estimador Forest Regressor"""
rscv = RandomForestRegressor(n_estimators=500, random_state=42, min_samples_split=2, min_samples_leaf=1, max_depth=10, bootstrap=True)
rscv_fit = rscv.fit(x_train, y_train)
predict = rscv.predict(x_test)
predict

array([ 4595.62768831,  4999.44895166,  7096.74120393, ...,
       14537.77167993, 18140.91366572, 14017.14020166])

In [49]:
y_test

3302     4599.0
4315     4999.0
1580     7099.0
4086    11336.0
3896    20189.0
         ...   
507      5399.0
5783    23176.0
2980    14586.0
5770    18083.0
4189    14014.0
Name: Price, Length: 2224, dtype: float64

In [50]:
print("Mean Absolute Error:", round(metrics.mean_absolute_error(y_test, predict), 4))
print("Mean Squared Error:", round(metrics.mean_squared_error(y_test, predict), 4))
print("Root Mean Squared Error:", round(np.sqrt(metrics.mean_squared_error(y_test, predict)), 4))
print("(R^2) Score:", round(metrics.r2_score(y_test, predict), 4))
print(f'Train Score : {rscv.score(x_train, y_train) * 100:.2f}% and Test Score : {rscv.score(x_test, y_test) * 100:.2f}% using Random Tree Regressor.')
errors = abs(predict - y_test)
mape = 100 * (errors / y_test)
accuracy = 100 - np.mean(mape)
print('Accuracy:', round(accuracy, 2), '%.') 

Mean Absolute Error: 239.4578
Mean Squared Error: 20978278.6188
Root Mean Squared Error: 4580.2051
(R^2) Score: 0.9896
Train Score : 99.90% and Test Score : 98.96% using Random Tree Regressor.
Accuracy: 99.02 %.
